In [1]:
import requests, json
from pprint import pprint
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from fixtures endpoint
fix = requests.get(base_url+'fixtures/').json()

# create fixtures dataframe
fixtures = pd.json_normalize(fix)


In [3]:
# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# create teams dataframe
teams = pd.json_normalize(r['teams'])


team_colours = ['#e20814', '#91beea', '#ca0b17', '#b60501', '#004a9b', '#021581', '#004b97', '#024593', '#282624', '#0c3e94', '#13428d', '#b30011', '#b1d4fa', '#dc1116', '#0d0805', '#f4031c', '#d20911', '#152055', '#7d2d3f', '#feb906']
teams['colours'] = team_colours

teams.head(20)

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,colours
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1250,1360,1260,1370,1240,1350,1,#e20814
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1110,1240,1080,1160,1140,1320,2,#91beea
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1140,1175,1080,1140,1190,1210,127,#ca0b17
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1080,1225,1070,1100,1090,1350,130,#b60501
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1110,1150,1080,1100,1140,1200,131,#004a9b
5,8,0,None,6,0,Chelsea,0,0,0,CHE,4,None,False,0,1255,1255,1150,1150,1360,1360,4,#021581
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1120,1125,1130,1140,1110,1110,6,#004b97
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1095,1125,1130,1150,1060,1100,7,#024593
8,54,0,None,9,0,Fulham,0,0,0,FUL,3,None,False,0,1125,1140,1140,1140,1110,1160,34,#282624
9,40,0,None,10,0,Ipswich,0,0,0,IPS,3,None,False,0,1075,1075,1070,1070,1080,1080,8,#0c3e94


In [4]:
# join fixtures to teams

df = pd.merge(
    left=fixtures,
    right=teams,
    left_on='team_a',
    right_on='id'
)

df2 = pd.merge(
    left=df,
    right=teams,
    left_on='team_h',
    right_on='id'
)

In [5]:
def get_team_record(team_id):
    team_name = teams.loc[teams.id == team_id, 'name'].values[0]

    record_df = df2[ (df2['team_h'] == team_id) | (df2['team_a'] == team_id) ]
    record_df = record_df.reset_index()
    #record_df[['name_y','team_h_score', 'team_a_score', 'name_x']].head(11)

    w = 0
    d = 0
    l = 0

    for index, row in record_df.iterrows():
        # print(row['name_y'],row['team_h_score'], row['team_a_score'], row['name_x'])
        if row['team_h'] == team_id and (row['team_h_score'] > row['team_a_score']):
            w += 1
            # print("home win")
        elif row['team_h'] == team_id and (row['team_h_score'] == row['team_a_score']):
            d += 1
            # print("home draw")
        elif row['team_h'] == team_id and (row['team_h_score'] < row['team_a_score']):
            l += 1
            # print("home loss")

        if row['team_a'] == team_id and (row['team_a_score'] > row['team_h_score']):
            w += 1
            # print("away win")
        elif row['team_a'] == team_id and (row['team_a_score'] == row['team_h_score']):
            d += 1
            # print("away draw")
        elif row['team_a'] == team_id and (row['team_a_score'] < row['team_h_score']):
            l += 1
            # print("away loss")
    
    remaining_df = df2[ (df2['finished'] == False) & ((df2['team_h'] == team_id) | (df2['team_a'] == team_id)) ]
    remaining_df = remaining_df.reset_index()
    left = int(38 - len(remaining_df))
    
    
    pts = int((w*3)+(d))
    ply = int(w+d+l)

    # print(f"{team_name} Record: Played: {ply}, Won: {w}, Draw: {d}, Loss: {l}, Points: {pts}")

    return pts, ply, left

points, played, remaining = get_team_record(15)

In [6]:
def get_remaining_fixtures(team_id):
    filtered_df = df2[ (df2['finished'] == False) & ((df2['team_h'] == team_id) | (df2['team_a'] == team_id)) ]
    filtered_df = filtered_df.reset_index()
    #filtered_df[['name_y','team_h_score', 'team_a_score', 'name_x']]

    remaining_fix = []
    fix_location = []
    opposition = []
    opp_name = []
    diff = []

    for index, row in filtered_df.iterrows():
        if row['team_h'] == team_id:
            location = "H"
            oppos = row['team_a']
            tname = row['name_x']
            opp_diff = row['team_h_difficulty']
        else:
            location = "A"
            oppos = row['team_h']
            tname = row['name_y']
            opp_diff = row['team_a_difficulty']

        date = row['kickoff_time'][5:10]
        left, right = date.split('-')
        newdate = f"{location} {right}-{left}"
        # print(newdate)
        remaining_fix.append(newdate)
        fix_location.append(location)
        opposition.append(oppos)
        opp_name.append(tname)
        diff.append(opp_diff)

    # print(remaining_fix)
    filtered_df['remaining_fixtures'] = remaining_fix
    filtered_df['fixture_location'] = fix_location
    filtered_df['opposition_id'] = opposition
    filtered_df['opposition_name'] = opp_name
    filtered_df['opposition_difficulty'] = diff

    filtered_df = filtered_df[['remaining_fixtures', 'fixture_location', 'opposition_id', 'opposition_name', 'opposition_difficulty']]


    return filtered_df

fix_left = get_remaining_fixtures(15)

fix_left.head()

# for index, fx in fix_left.iterrows():
    # print(fx['opposition_id'])

,remaining_fixtures,fixture_location,opposition_id,opposition_name,opposition_difficulty
0,H 15-01,H,20,Wolves,2
1,H 18-01,H,3,Bournemouth,3
2,A 25-01,A,17,Southampton,2
3,H 01-02,H,9,Fulham,3
4,A 15-02,A,13,Man City,4
